In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.spatial.distance as dist
import matplotlib as mpl
import iblofunmatch.inter as ibfm
output_dir="output"

from navground import sim, core

### Corridor trajectories experiment

Corridor experiment with points converging to horizontal trajectories going left and right.

In [ ]:
length = 8.0
num_steps = 5000
width=1.1
yaml = f"""
steps: 3000
time_step: 0.1
save_directory: ''
record_pose: true
record_twist: true
scenario:
  type: Corridor
  length: {length}
  width: {width} 
  groups:
    -
      type: thymio
      number: 76
      radius: 0.08
      control_period: 0.1
      speed_tolerance: 0.02
      kinematics:
        type: 2WDiff
        wheel_axis: 0.094
        max_speed: 0.166
      behavior:
        type: HL
        optimal_speed: 0.12
        horizon: 5.0
        safety_margin: 0.02
      state_estimation:
        type: Bounded
        range: 5.0
"""
experiment = sim.load_experiment(yaml)
experiment.run()

In [ ]:
run = experiment.runs[0]
ps = run.poses[:,:,[0,1]]
twists = run.twists[:,:,:2] # ignore angular speeds

In [ ]:
weight = 5

In [ ]:
def trajectory_corridor_distance_weighted_velocities(positions, velocities, weight, length):
    assert(len(positions)>0)
    assert(len(positions)==len(velocities))
    positions_velocities_list = []
    for idx, points in enumerate(positions):
        positions_velocities_list.append(np.hstack((points, velocities[idx]*weight)))
    distances_list = []
    for idx, points_vel in enumerate(positions_velocities_list):
        # Compare trajectories at same time
        points_vel_compare = positions_velocities_list[idx]
        dist_0 = dist.cdist(points_vel, points_vel_compare, "minkowski", p=2)
        shift_points_vel_compare = np.array(points_vel_compare)
        shift_points_vel_compare[shift_points_vel_compare[:,0]<length/2]+=[length, 0, 0, 0]
        shift_points_vel = np.array(points_vel)
        shift_points_vel[shift_points_vel[:,0]<length/2]+=[length, 0, 0, 0]
        dist_1 = dist.cdist(shift_points_vel, shift_points_vel_compare, "minkowski", p=2)
        distances_list.append(np.minimum(dist_0, dist_1))
    # end for
    distances_arr = np.array(distances_list)
    return np.min(distances_arr, axis=0)

In [ ]:
def plot_matching_diagram_trajectories(positions, velocities, weight, length, ax, color="blue"):
    half_step = int(len(positions)/2)+1
    # Compute X, Z barcodes and matching
    Dist_X = trajectory_corridor_distance_weighted_velocities(positions[:half_step], velocities[:half_step], weight, length)
    Dist_Z = trajectory_corridor_distance_weighted_velocities(positions, velocities, weight, length)
    idx_S = list(range(Dist_X.shape[0]))
    # Compute matching from X to Z
    ibfm_out = ibfm.get_IBloFunMatch_subset(Dist_X, Dist_Z, idx_S, output_dir, max_rad=-1, num_it=1, store_0_pm=True, points=False, max_dim=1)
    # Plot 0 persistence diagram of matching 
    match_diagram = []
    for idx, bar_X in enumerate(ibfm_out["S_barcode_0"]):
        idx_match = ibfm_out["induced_matching_0"][idx]
        bar_Z = ibfm_out["X_barcode_0"][idx_match]
        match_diagram.append([bar_X[1], bar_Z[1]])
    # end for
    match_diagram = np.array(match_diagram)
    # Plot matching diagram
    ax.scatter(match_diagram[:,0], match_diagram[:,1], color=color)
    ax.plot([0,np.max(match_diagram)*1.1], [0,np.max(match_diagram)*1.1], color="gray")

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
steplist = list(range(500,900, 10))
shift_time = 40
for idx, start_step in enumerate(steplist):
    plot_matching_diagram_trajectories(ps[start_step:start_step+shift_time], twists[start_step:start_step+shift_time], weight, length, ax, color=mpl.colormaps["GnBu"](idx/len(steplist)))


In [ ]:
def plot_sequence(X_list, ax, mark_points=[]):
    # Plot figure
    X_old = X_list[0]
    ax.scatter(X_old[:,0], X_old[:,1], s=20, marker="o", color=mpl.colormaps["RdBu"](1/(len(X_list)+1)), zorder=2)
    for idx, X in enumerate(X_list[1:]):
        ax.scatter(X[:,0], X[:,1], s=20, marker="o", color=mpl.colormaps["RdBu"]((idx+1)/(len(X_list)+1)), zorder=2, label="X")
        # for edge in zip(X, X_old):
        #     edge_pts = np.array(edge)
        #     ax.plot(edge_pts[:,0], edge_pts[:,1], color="gray", zorder=1)
        if len(mark_points)>0:
            mark_X = X[mark_points]
            ax.scatter(mark_X[:,0], mark_X[:,1], s=20, marker="+", color="red", zorder=3)
        X_old = X
    #end for 

In [ ]:
start_step=800
fig, ax = plt.subplots(figsize=(5, 5))
plot_matching_diagram_trajectories(ps[start_step:start_step+shift_time], twists[start_step:start_step+shift_time], weight, length, ax, color="blue")
positions = ps[start_step:start_step+shift_time]
velocities = twists[start_step:start_step+shift_time]
half_step = int(shift_time/2)+1
Dist_X = trajectory_corridor_distance_weighted_velocities(positions[:half_step], velocities[:half_step], weight, length)
Dist_Y = trajectory_corridor_distance_weighted_velocities(positions, velocities, weight, length)
Dist_Z = np.minimum(Dist_X, Dist_Y)
idx_S = list(range(Dist_X.shape[0]))
# Compute matching from X to Z
ibfm_out = ibfm.get_IBloFunMatch_subset(Dist_X, Dist_Z, idx_S, output_dir, max_rad=-1, num_it=1, store_0_pm=True, points=False, max_dim=1)
# put persistence pairs together
match_diagram = []
for idx, bar_X in enumerate(ibfm_out["S_barcode_0"]):
    idx_match = ibfm_out["induced_matching_0"][idx]
    bar_Z = ibfm_out["X_barcode_0"][idx_match]
    match_diagram.append([bar_X[1], bar_Z[1]])
# end for
match_diagram = np.array(match_diagram)

print(np.array(match_diagram))
print(ibfm_out["S_reps_0"])

In [ ]:
X_seq = ps[list(range(start_step, start_step+shift_time+1, 2))]
len(X_seq)
fig, ax = plt.subplots(figsize=(10, 5))
ax.set_aspect("equal")
plot_sequence(X_seq, ax, mark_points=[1])